# 3D Face Mesh

In [ ]:
!pip install mediapipe opencv-python

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import necessary libraries
import cv2
import mediapipe as mp
from google.colab.patches import cv2_imshow
from google.colab import drive
import time

# Define a class for processing FaceMesh
class FaceMeshProcessor:
    def __init__(self, input_source, output_file=None):
        # Initialize class variables
        self.input_source = input_source
        self.output_file = output_file
        self.video_capture = cv2.VideoCapture(input_source)
        self.frame_rate = 30

        # Set up FaceMesh and drawing utilities from Mediapipe
        self.mp_face_mesh = mp.solutions.face_mesh
        self.mp_drawing = mp.solutions.drawing_utils

        # Initialize FaceMesh with specified parameters
        with self.mp_face_mesh.FaceMesh(
            max_num_faces=5,
            refine_landmarks=True,
            min_detection_confidence=0.7,
            min_tracking_confidence=0.7,
        ) as self.face_mesh:
            self.mp_drawing_styles = mp.solutions.drawing_styles
            # Check if the input is an image and call the appropriate processing method
            if self.is_image():
                self.process_image()
            else:
                self.process_video()

    # Check if the input source is an image
    def is_image(self):
        return self.input_source.lower().endswith(('.png', '.jpg', '.jpeg'))

    # Draw FaceMesh on the frame
    def draw_face_mesh(self, frame, face_landmarks):
        # Draw different components of the face mesh
        self.mp_drawing.draw_landmarks(
            image=frame,
            landmark_list=face_landmarks,
            connections=self.mp_face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=self.mp_drawing_styles.get_default_face_mesh_tesselation_style()
        )
        self.mp_drawing.draw_landmarks(
            image=frame,
            landmark_list=face_landmarks,
            connections=self.mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=self.mp_drawing_styles.get_default_face_mesh_contours_style()
        )
        self.mp_drawing.draw_landmarks(
            image=frame,
            landmark_list=face_landmarks,
            connections=self.mp_face_mesh.FACEMESH_IRISES,
            landmark_drawing_spec=None,
            connection_drawing_spec=self.mp_drawing_styles.get_default_face_mesh_iris_connections_style()
        )

    # Process video input
    def process_video(self):
        # Get video dimensions
        width = int(self.video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(self.video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Initialize video writer if output file is provided
        if self.output_file:
            fourcc = cv2.VideoWriter_fourcc(*'XVID')
            out = cv2.VideoWriter(self.output_file, fourcc, self.frame_rate, (width, height))

        # Initialize variables for FPS calculation
        frame_count = 0
        start_time = time.time()

        # Process each frame of the video
        while self.video_capture.isOpened():
            isSuccess, frame = self.video_capture.read()
            if not isSuccess:
                break

            # Convert frame to RGB for processing
            frame.flags.writeable = False
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = self.face_mesh.process(frame)

            # Convert frame back to BGR for display
            frame.flags.writeable = True
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

            # Draw FaceMesh on the frame if landmarks are detected
            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    self.draw_face_mesh(frame, face_landmarks)

            # Calculate and display FPS
            frame_count += 1
            elapsed_time = time.time() - start_time
            fps = frame_count / elapsed_time
            cv2.putText(frame, f'FPS: {fps:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            # Display the frame
            cv2_imshow(cv2.flip(frame, 1))

            # Write frame to output video if specified
            if self.output_file:
                out.write(frame)

            # Break the loop if 'Esc' key is pressed
            if cv2.waitKey(int(1000 / self.frame_rate)) & 0xFF == 27:
                break

        # Release resources
        self.video_capture.release()
        if self.output_file:
            out.release()
        cv2.destroyAllWindows()

    # Process image input
    def process_image(self):
        # Read image and convert to RGB
        frame = cv2.imread(self.input_source)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.face_mesh.process(frame)

        # Draw FaceMesh on the image if landmarks are detected
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                self.draw_face_mesh(frame, face_landmarks)

        # Display the image
        cv2_imshow(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

        # Save the image as a JPEG file if output file is specified
        if self.output_file:
            output_path = self.output_file.replace('.avi', '.jpg')
            cv2.imwrite(output_path, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
        cv2.destroyAllWindows()

# Mount Google Drive
drive.mount('/content/drive')

# Define input and output paths
if __name__ == "__main__":
    input_source = '/content/drive/MyDrive/3D-FaceMesh/faces/worker.mp4'
    output_file = '/content/drive/MyDrive/3D-FaceMesh/Output/output_video3.avi'

    # Create FaceMeshProcessor instance and process input
    face_mesh_processor = FaceMeshProcessor(input_source, output_file)